In [1]:
# Declaring path variables to make Basemap work in jupyter hub. Make sure that the conda environment path matches yours
import os
os.environ['PATH'] = '/home/jovyan/my-conda-envs/qiime2-2019.1/bin:' + os.environ['PATH']
os.environ['PROJ_LIB'] = '/home/jovyan/my-conda-envs/qiime2-2019.1/share/proj'

In [2]:
# General Tool Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from mpl_toolkits.basemap import Basemap
import biota
import biota.plot as bpl
import seaborn as sns
import collections
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
import math

from biota.database import (get_samples,
                            get_sample_data,
                            get_pg_connection,
                            get_data)
from biota.util import (rarefy,
                        make_tree,
                        get_dms,
                        filter_dm_and_map,
                        get_tax_table)
from IPython.core.display import display, HTML
from pickle import dump, load

import lasio

# SQL Query imports
from sqlalchemy import create_engine, engine, inspect

from functools import partial

from ipyparallel import Client
from sourcetracker import gibbs

# Clustering imports
from scipy.spatial.distance import pdist
from scipy.cluster import hierarchy
from skbio.stats.ordination import pcoa

from biota.util import assign_formations


%matplotlib inline

In [3]:
df = pd.read_csv('PDX_DAILY_J752801_20190606_1H.csv')

In [19]:
ProdStart = df[df['Cum Oil Prod.'] > 3].index[0]

56

In [25]:
# Define Cums in days since online
Mo3Date = ProdStart + 90
Mo6Date = ProdStart + 180
Mo9Date = ProdStart + 270

In [26]:
Mo3Cum = df.loc[Mo3Date,'Cum Oil Prod.']
Mo6Cum = df.loc[Mo3Date,'Cum Oil Prod.']
Mo9Cum = df.loc[Mo3Date,'Cum Oil Prod.']

In [27]:
print (Mo3Cum)
print (Mo3Cum)
print (Mo3Cum)

74019.89


In [16]:
d = df.loc[28,'Date and Time']

AttributeError: 'str' object has no attribute 'copy'

In [8]:
df.head()

,Well,Date and Time,Elapsed Time,Choke,Tubing Pressure,Casing Pressure,Gas Rate,Prod.,Oil Rate,Cum Oil Prod.,Water Rate,Cum Water,Remaining,Recovered,Water
0,Silvertip 1H,9/26/2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,644487.97,0.0,NaN
1,Silvertip 1H,9/27/2018,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,644487.97,0.0,NaN
2,Silvertip 1H,9/28/2018,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,644487.97,0.0,NaN
3,Silvertip 1H,9/29/2018,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,644487.97,0.0,NaN
4,Silvertip 1H,9/30/2018,96.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,644487.97,0.0,NaN


In [ ]:
#Create List of filenames
file_names = ['Silvertip A 1H.csv','Silvertip A 2H.csv','Silvertip A 3H.csv','Silvertip A 4H.csv','Silvertip A 5H.csv','Silvertip A 6H.csv','Silvertip A 7H.csv','Silvertip A 8H.csv',\
              'Silvertip A 9H.csv','Silvertip A 10H.csv','Silvertip B 2H.csv','Silvertip B 3H.csv']
well_names = ['STA1','STA2','STA3','STA4','STA5','STA6','STA7','STA8','STA9','STA10','STB2','STB3']

In [ ]:
AB_wells = ['STA1','STA2','STA3','STA4','STA5','STA6','STA7','STA8','STA9','STA10','STB2','STB3']

In [ ]:
AllWells = pd.read_csv('SILVERTIP_ALL_WELLS_IPSO_GENERAL_DATA.csv')

In [ ]:
ABWells = AllWells.iloc[19:32,:].copy()

In [ ]:
#Define comp_order 
online_order = {'SILVERTIP 76-7 UNIT A 10H' : 8,'SILVERTIP 76-7 UNIT A 9H' : 7, 'SILVERTIP 76-7 UNIT A 8H' : 4, 'SILVERTIP 76-7 UNIT A 7H' : 6, \
 'SILVERTIP 76-7 UNIT A 6H' : 11, 'SILVERTIP 76-7 UNIT A 5H' : 3, 'SILVERTIP 76-7 UNIT A 4H' : 9, 'SILVERTIP 76-7 UNIT A 3H' : 2, \
 'SILVERTIP 76-7 UNIT A 2H' : 1, 'SILVERTIP 76-7 UNIT A 1H' : 5, 'SILVERTIP 76-7 UNIT B 1H' : 0, 'SILVERTIP 76-7 UNIT B 2H' : 10, \
 'SILVERTIP 76-7 UNIT B 3H' : 12}

In [ ]:
#Define frac_order 
frac_order = {'SILVERTIP 76-7 UNIT A 10H' : 8,'SILVERTIP 76-7 UNIT A 9H' : 7, 'SILVERTIP 76-7 UNIT A 8H' : 4, 'SILVERTIP 76-7 UNIT A 7H' : 6, \
 'SILVERTIP 76-7 UNIT A 6H' : 11, 'SILVERTIP 76-7 UNIT A 5H' : 3, 'SILVERTIP 76-7 UNIT A 4H' : 9, 'SILVERTIP 76-7 UNIT A 3H' : 2, \
 'SILVERTIP 76-7 UNIT A 2H' : 1, 'SILVERTIP 76-7 UNIT A 1H' : 5, 'SILVERTIP 76-7 UNIT B 1H' : 0, 'SILVERTIP 76-7 UNIT B 2H' : 10, \
 'SILVERTIP 76-7 UNIT B 3H' : 12}

In [ ]:
ABWells['Order Online']= ABWells['Well Name'].map(online_order)

In [ ]:
comp_para = ['Well Name','Order Online','Cumulative Oil Production (Current)','Cumulative Gas Production (Current)','Cumulative Water Production (Current)','Cumulative Liquid Production (Current)','Cumulative BOE Production (Current)']

In [ ]:
ABComps = ABWells [comp_para].copy()

In [ ]:
ABComps.head()

In [ ]:
ABWells.head()

In [ ]:
AllWells.head()

In [ ]:
#Create dictionary of dataframes for each well 
dfs = {}
for i in range(len(well_names)):
    well = well_names[i]
    filename = file_names[i]
    dfs[well] = pd.read_csv(filename)

In [ ]:
#Rearrange structure into easily plotted data
for well in well_names:
    dfs[well] = dfs[well].T
    dfs[well].columns = dfs[well].iloc[0]
    dfs[well].insert(0, 'Stages', value = dfs[well].index)
    dfs[well] = dfs[well].set_index('Well Name').copy()
    dfs[well] = dfs[well].drop(dfs[well].index[0]).copy()
    dfs[well] = dfs[well].astype(float)

In [ ]:
for well in well_names:
    filename = 'C:/Users/jsawa/Documents/Biota/'+ well + '.csv'
    print(filename)

In [ ]:
for well in well_names:
    #filename = well + '.csv'
    filename = 'C:/Users/jsawa/Documents/Biota/'+ well + '.csv'
    dfs[well].to_csv(rfilename)

In [ ]:
dfs['STA3'].head()

In [ ]:
dfs['STA3'].to_csv('Test.csv')

In [ ]:
#Set list of relevant completion parameters
Comp_vars = ['Total Proppant','Ave Rate','Ave Pres', 'Num Perfs','Total Fluid','Break Pres']


In [ ]:
#Obtain list of all completion parameters
All_comp_vars = dfs['STA1'].columns
All_comp_vars

In [ ]:
for col,well in enumerate(well_names):
    print(col,well)
    #fig = plt.figure()
    #ax = sns.lineplot(x = dfs[well]['Stages'], y = dfs[well].loc[col], label = well)

In [ ]:
#Plot completion parameter for each well
for i in range(len(Comp_vars)):
    fig = plt.figure()
    plt.figure(figsize=(15,10))
    for well in dfs:
        #if well != 'STA6':
        ax = sns.lineplot(x = dfs[well]['Stages'], y = dfs[well][Comp_vars[i]], label = well)
        

In [ ]:
T_vars = ['Ave Rate','Ave Pres']

for i in range(len(T_vars)):
    fig = plt.figure()
    for well in dfs:
        if well == 'STA6':
            ax = sns.lineplot(x = dfs[well]['Stages'], y = (dfs[well]['Ave Rate'],dfs[well]['Ave Pres']), label = well)
        

In [ ]:
#ProdData_T

In [ ]:
#ProdData_T.reset_index()
#ProdData_T["Stage"]

ax = sns.lineplot(x="Stage", y="Average Rate, bpm", data=ProdData_T)

